Training notebook for the 5 minute grid demand forecasting

In [ ]:
import os
import sys
from pathlib import Path

from hydra import compose, initialize
from omegaconf import OmegaConf

%load_ext autoreload
%autoreload 2

src_folder = os.path.join(Path(os.getcwd()).parent.parent, "src")
sys.path.insert(0, src_folder)

from grid_demand.data_processing import generate_feature_table, onload
from grid_demand.training import run

config_path = "../../conf"
environment = "local"
horizon = "1d"
experiment_id = 969986715520194561

with initialize(version_base=None, config_path=config_path):
    cfg = compose(
        config_name="config.yaml",
        overrides=[
            f"environment={environment}",
            f"++grid_demand.mlflow.experiment_id={experiment_id}",
            f"grid_demand={horizon}",
        ],
    )
    print(OmegaConf.to_yaml(cfg))


Generating the feature table

In [ ]:
df = onload(cfg)
df = generate_feature_table(df, cfg)
df.head()

Train function

In [ ]:
df = df[df["date"] > "2023-02-01"].copy()
run(df, cfg)